In [1]:
import pandas as pd
import scipy as sp
import scipy.sparse
import scipy.sparse.linalg
from sklearn.metrics import confusion_matrix

In [6]:
import numpy as np


def get_distances(centroids, pixels):
    distances = []
    for centroid in centroids:
        distances.append(np.linalg.norm((centroid - pixels), axis=1))
    return distances


def join_distances(distances):
    distances = [x.reshape((x.shape[0], 1)) for x in distances]
    distances_joined = np.concatenate(distances, axis=1)
    return distances_joined


def get_min_distance(distances_joined):
    min_distances = np.min(distances_joined, axis=1)
    return min_distances


def get_probabilities(min_distances):
    dis_sq = np.square(min_distances)
    a = np.sum(dis_sq)
    p = np.multiply(1 / a, dis_sq)

    return p


def get_centroid(centroids, pixels):
    distances = get_distances(centroids, pixels)
    distances_joined = join_distances(distances)
    min_distances = get_min_distance(distances_joined)
    p = get_probabilities(min_distances)
    idxs = [x for x in range(pixels.shape[0])]
    idx = np.random.choice(idxs, 1, p=p)
    return pixels[idx]


def k_plus_plus(pixels, K):
    pixels_copy = pixels.copy()
    np.random.shuffle(pixels_copy)
    
    centroids = pixels_copy[0, :].reshape(1, pixels_copy.shape[1])

    while len(centroids) < K:
        new_centroid = get_centroid(centroids, pixels).reshape(1, pixels_copy.shape[1])
        centroids = np.append(centroids, new_centroid, axis=0)

    return centroids


def get_classes(centroids, pixels):
    distances = get_distances(centroids, pixels)
    distances_joined = join_distances(distances)
    classes = np.argmin(distances_joined, axis=1)
    return classes


def get_new_centroids(centroids, pixels):
    classes = get_classes(centroids, pixels)
    new_centroids = np.zeros(shape=centroids.shape)
    for i in range(len(centroids)):
        centroid = np.mean(pixels[classes == i], axis=0)
        new_centroids[i] = centroid

    return new_centroids


def get_centroids(pixels, K):
    centroids = k_plus_plus(pixels, K)
    c = np.full(centroids.shape, np.inf)
    centroid_distances = np.linalg.norm((centroids - c), axis=1)
    while np.max(centroid_distances) > 1:
        new_centroids = get_new_centroids(centroids, pixels)
        centroid_distances = np.linalg.norm((centroids - new_centroids), axis=1)
        centroids = new_centroids
    return centroids


def my_kmeans(pixels, K):
    centroids = get_centroids(pixels, K)
    classes = get_classes(centroids, pixels)
    return classes, centroids



In [2]:
df = pd.read_csv('edges.txt', sep="\t", header=None, names=['org','dest'])

In [3]:
nodes = pd.read_csv('nodes.txt', sep="\t", header=None, names=['blog','class', "site"])


In [4]:
df['weight']=1
df['org'] = df['org']-1
df['dest'] = df['dest']-1

In [7]:
# Step 1: represent graph as adjacency matrix
A = sp.sparse.coo_matrix((df['weight'],
                          (df['org'], df['dest'])),
                         shape=(1490, 1490))
#Step 2: form a special matrix L=D-A, the graph Laplacian
l,d = scipy.sparse.csgraph.laplacian(A, return_diag=True)
D = sp.sparse.diags(d)
L = D-A
# Step 3: compute k eigenvectors, v1,v2...vk corresponding to the k smallest eigenvalues (k ≪ m)
w,Z = sp.sparse.linalg.eigs(L, k=2, which='SR')
# Step 4: run kmeans algorithm on Z = (v1,v2,...vk) by treating each row as a new data point
classes,centroids = my_kmeans(Z, 2)

/home/jfftilton/anaconda3/envs/cv/lib/python3.6/site-packages/numpy/linalg/linalg.py:2480: RuntimeWarning: invalid value encountered in multiply
  s = (x.conj() * x).real
/home/jfftilton/.local/lib/python3.6/site-packages/ipykernel_launcher.py:65: ComplexWarning: Casting complex values to real discards the imaginary part


In [172]:
y_true = [int(float(x)) for x in nodes["class"]]
y_pred = classes
y_pred2 = [1 if x ==0 else 0 for x in classes]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
(tn, fp, fn, tp)

(101, 657, 57, 675)

In [173]:
cm = pd.DataFrame({"Predicted No":[tn,fn], "Predicted Yes":[fp,tp]})
cm.index = ["Actual No", "Actual Yes"]
cm

,Predicted No,Predicted Yes
Actual No,101,657
Actual Yes,57,675


In [174]:
false_classification_rate = (fp+fn)/1490 * 100
false_classification_rate

47.91946308724832

In [21]:
x = A-np.zeros(A.shape)

In [24]:
x.T

matrix([[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])